In [ ]:
import numpy as np
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force
import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
import importlib
import matplotlib.animation as animation

import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os, datetime
from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential



coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
# 4. Szilard engine test

# parameters

In [ ]:

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

C_factor = 100
L_factor = 0.6
# L_factor = 1
I_m_factor = 0
I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
R_1, R_2 = 371, 371                                # ohm
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 

quick_doubler = lambda x1, x2: np.hstack([np.array([x1] * 2), np.array([x2]*2)])
I_p, I_m = quick_doubler(I_p_1, I_p_2), quick_doubler(I_m_1, I_m_2)

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)


U0_1 = m_c * x_c**2 / t_c**2
U0_2 = m_2 * x_c**2 / t_c**2
kappa_1, kappa_2, kappa_3, kappa_4 = k_BT/U0_1, k_BT/U0_1, k_BT/U0_1, k_BT/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/1000
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "Experiment 4a (2024/2/22): Effect of L and the three peak, L_factor = 1"



# define potential

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
print(U0_1/k_BT, U0_2/k_BT)

In [ ]:
np.linspace(0.1, 2.1, 11)

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_dcx_off = 0
phi_2_dcx_off = 0
M_12_off = 0


# bookmark
# phi_1_dcx_on = 2.8
# phi_2_dcx_on = 2.8
# M_12_on = -0.6
phi_x_on = -0.0
phi_1_x_on = 0
phi_2_x_on = 0
phi_1_dcx_on = 2.8
phi_2_dcx_on = 2.8
M_12_on = -0.3
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_on,  "phi_2_x": phi_2_x_on,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



In [ ]:
zeroDissipation = False
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameter'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "phi_1_dcx_on": phi_1_dcx_on, "phi_2_dcx_on": phi_2_dcx_on, "M_12_on": M_12_on
}
    
TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 0

# bookmark
# initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index-1]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
TR_initial_condition[0]

rest = lambda t: {"duration": t, "name":"rest"}
# protocol_list = [
#     # forward
#     {"duration":50, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1.1) mix in y direction"},
#     {"duration":200, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1.2) mix in y direction"},
#     {"duration":60, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt in y"},
#     {"duration":100, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
#     {"duration":60, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5.1) mix in x direction"}, 
#     {"duration":200, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5.2) mix in x direction"}, 
#     {"duration":50, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
# ]

protocol_list = [
    # forward
    # {"duration":50, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1.1) mix in y direction"},
    {"duration":400, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1.2) mix in y direction"},
    {"duration":200, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt in y"},
    # {"duration":100, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,   "name":"(3) raise the barrier"},
    # {"duration":50, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5, "M_12": M_12_on * 0.5,  "name":"(6) 4 well potential"},  
    # {"duration":300, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5, "M_12": M_12_on * -.5,  "name":"(6) 4 well potential"}, 
    {"duration":200, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
]

protocol_time_array = [item["duration"] for item in protocol_list]
protocol_time_array.insert(0, 0)
protocol_time_array = np.cumsum(protocol_time_array)
print(protocol_time_array)

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

# comp_protocol.protocol_array[0].interpolation = "inverted_quadratic"

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""
cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, measure_all_states=True)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(extra_constraint=extra_constraint_00_and_11_only)
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state


# potential shape check

In [ ]:
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
import importlib
general_analysis_tool = importlib.reload(general_analysis_tool)
# minimum_point_data = minimum_value_of_potential.obtain_minimum_points(cfqr, protocol_time_array, beta_1, beta_2, d_beta_1, d_beta_2)
# minimum_points = [d['coordinate'] for d in minimum_point_data]
# minimum_points_potential = np.array([d['min_E'] for d in minimum_point_data])
# energy_profile_array = []
# print(minimum_points, minimum_points_potential)

In [ ]:
checkInitial = False
if checkInitial:
    cfqr.show_initial_state()
    
    
    general_analysis_tool.showParametersVsTime(cfqr, protocol_time_array)

# actual simulations

In [ ]:
manual_domain=[np.array([-5, -5])/time_scale_factor, np.array([5, 5])/time_scale_factor]
mapping_state_1_to_state_2_dict = {"00": ["00", "10"], "01": ["00", "10"], "10": ["01", "11"], "11": ["01", "11"]}

In [ ]:
simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_saved, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = np.s_[::], measure_all_states=True)

# Fidelity

In [ ]:
simResult["fidelity"]

# work analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))

print(params['comment'])
print("sim_id = ", simResult['simulation_data'])
print(f"N = {params['N']}, dt = {params['dt']}")
print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}, gamma = {gamma}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')


In [ ]:

step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
skip_step = int(len(step_array) * 0.01)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

coarse_step_array = step_array[::skip_step]
coarse_step_array = np.append(coarse_step_array, step_array[-1])
coarse_work_mean_array = work_mean[::skip_step]
coarse_work_mean_array = np.append(coarse_work_mean_array, work_mean[-1])
coarse_work_std_array = np.append(work_std[::skip_step], work_std[-1])

plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array)
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', label = 'axvline - full height')
    # plt.text('2017-07-02 16:30:00',0.005,'BigNews1',rotation=90,va='top')
    plt.annotate(f"{i+1}", 
            xy=(t, 4.5),  
            # xytext=(quantile_75, 2.05*1e9),
            color='red', 
            ha='center', 
            size=16)

first_half = work_mean[0: len(work_mean)//2]
second_half = work_mean[len(work_mean)//2:][::-1]
abs_error = np.abs(first_half - second_half)
# plt.plot(step_array[0:len(step_array)//2], abs_error)
print(max(abs_error), min(abs_error))
print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:
minimum_value_of_potential = importlib.reload(minimum_value_of_potential)

In [ ]:
potential_type_and_time_array = list(zip(protocol_time_array, ["four_well", "mix_in_y", "mix_in_y", "conditional_tilt_in_y", "conditional_tilt_in_x", "mix_in_x", "mix_in_x", "four_well"]))
minimum_point_data_array = minimum_value_of_potential.obtain_minimum_points(cfqr, potential_type_and_time_array, beta_1, beta_2, d_beta_1, d_beta_2, verbose = False)

# offset_value = np.array([d['min_E'] for d in minimum_point_data_array])
offset_value = np.array([ 99.89691102, 105.27414166,  56.69629202,  52.14918188, 52.14918188,  56.69629202, 105.27414166,  99.89691102])
offset_value = offset_value[1:] - offset_value[:-1]

In [ ]:
offset_value

In [ ]:
# work_done_of_each_step = general_analysis_tool.get_work_done_from_time_array(params, work_mean, protocol_time_array, offset_value)

In [ ]:

# target_time_index = np.array(protocol_time_array)/params['dt']
# target_time_index = target_time_index.astype("int") - 1

# work_done_array = [work_mean[i] for i in target_time_index[1:]]
# work_done_array.insert(0, 0)
# work_done_of_each_step = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]

# work_done_after_offset = work_done_of_each_step - offset_value


In [ ]:
# offset_value

In [ ]:
# print("Third way to split the steps: (E, 0-1), (M. 2), (C: 3-6)")
# print(f"work_done = {work_done_of_each_step}" )
# print(f"work_offset = {work_done_after_offset}")
# print("measurement + reset = ", work_done_after_offset[0] + work_done_after_offset[1] + work_done_after_offset[2] )
# print("control = ", work_done_after_offset[3] + work_done_after_offset[4] + work_done_after_offset[5] + work_done_after_offset[6])
# print(sum(work_done_after_offset))

In [ ]:

# print("First way to split the steps: (E, 5-6), (M, 0-2), (C: 3-4)")
# print(f"work_done = {work_done_of_each_step}" )
# print(f"work_offset = {work_done_after_offset}")
# print("measurement + reset = ", work_done_after_offset[0] + work_done_after_offset[1] + work_done_after_offset[2] + work_done_after_offset[5] + work_done_after_offset[6] )
# print("control = ", work_done_after_offset[3] + work_done_after_offset[4] )
# print(sum(work_done_after_offset))

# animations

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 20000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
def plot_work_dist_with_time(cfqr, skipNumber = 2000):
    work_dist_time_array = cfqr.sim.work_dist_time_array
    step_time_array = np.array(cfqr.sim.output.all_state["step_indices"]) * params['dt']
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    _skipped_work_dist_time_array = work_dist_time_array[::skipNumber]
    _skipped_step_time_array = step_time_array[::skipNumber]
    
    def animate(i):
        ax.clear()
        # Get the point from the points list at index i
        _dist = _skipped_work_dist_time_array[i]
        work_min = np.min(_dist) - 5
        work_max = np.max(_dist) + 5
        bins = np.linspace(work_min, work_max, 50)

        # Plot that point using the x and y coordinates
        ax.hist(_dist, bins = bins)
        ax.set_title(f"t = {_skipped_step_time_array[i]}")
        # Set the x and y axis to display a fixed range
    ani = animation.FuncAnimation(fig, animate, frames=len(_skipped_work_dist_time_array),
                        interval=100, repeat=False)
    plt.close()
    return ani

In [ ]:
work_dist_time_array = simResult['cfqr'].sim.work_dist_time_array

In [ ]:
html = display.HTML(plot_work_dist_with_time(simResult['cfqr']).to_html5_video())


In [ ]:
display.display(html)


# phi_1dc, phi_2dc and gamma

In [ ]:
general_analysis_tool = importlib.reload(general_analysis_tool)
general_analysis_tool.show_phi_dc_with_time(simResult['cfqr'], params)

# save result

In [ ]:
params['comment'] = "Experiment 5c (2024/2/24): try to use M_12 = -0.3, for the erasure process"


In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery")

# save results

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False)

In [ ]:
simResult['simulation_data']